In [1]:
import mne
import os
import numpy as np
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
from mne.preprocessing import read_ica

# -------------------------
# CONFIG (EDIT THESE)
# -------------------------
BASE_DIR = r"rEEG"
SUBJ = "sub-095"
SID = "095"

RAW_PATH = rf"{BASE_DIR}\{SUBJ}\preICA{SID}_raw.fif"
ICA_PATH = rf"{BASE_DIR}\{SUBJ}\ica-{SID}-variance_compica.fif"
OUT_PATH = rf"{BASE_DIR}\{SUBJ}\postICA{SID}_raw.fif"

# -------------------------
# LOAD DATA
# -------------------------
raw = mne.io.read_raw_fif(RAW_PATH, preload=True)
ica = read_ica(ICA_PATH)

# -------------------------
# VIEW ICA SOURCES
# -------------------------
ica.plot_sources(raw, show_scrollbars=True)
plt.show()

# -------------------------
# MANUAL IC ENTRY (from Notion)
# -------------------------
reject_str = input(
    "Enter ICs to reject (comma-separated, e.g. 0,1,4) or press Enter for none: "
)

if reject_str.strip():
    ica.exclude = [int(x) for x in reject_str.split(",")]
else:
    ica.exclude = []

# -------------------------
# APPLY ICA
# -------------------------
raw_clean = raw.copy()
ica.apply(raw_clean)

# -------------------------
# SAVE CLEANED RAW
# -------------------------
raw_clean.save(OUT_PATH, overwrite=True)
print(f"Saved cleaned raw → {OUT_PATH}")

# ======================================================
#               SANITY CHECKS (DO ONCE)
# ======================================================

# 1. TIME SERIES COMPARISON
raw.plot(n_channels=20, block=True)
plt.suptitle("Raw BEFORE ICA")
plt.show()

raw_clean.plot(n_channels=20, block=True)
plt.suptitle("Raw AFTER ICA")
plt.show()

# 2. POWER SPECTRAL DENSITY
psd_raw = raw.compute_psd(fmax=45)
psd_clean = raw_clean.compute_psd(fmax=45)

psd_raw.plot()
plt.title("PSD BEFORE ICA")
plt.show()

psd_clean.plot()
plt.title("PSD AFTER ICA")
plt.show()

# 3. CHECK REJECTED IC PROPERTIES
# if len(ica.exclude) > 0:
#     ica.plot_properties(raw, picks=ica.exclude)

# 4. CHANNEL VARIANCE CHECK
var_raw = np.var(raw.get_data(), axis=1)
var_clean = np.var(raw_clean.get_data(), axis=1)

plt.figure()
plt.plot(var_raw, label="Raw", alpha=0.7)
plt.plot(var_clean, label="Cleaned", alpha=0.7)
plt.legend()
plt.title("Channel Variance Comparison")
plt.show()


Opening raw data file rEEG\sub-095\preICA095_raw.fif...
    Range : 0 ... 30834 =      0.000 ...   123.336 secs
Ready.
Reading 0 ... 30834  =      0.000 ...   123.336 secs...
Reading c:\Users\User\Documents\EEG_Project\rEEG\sub-095\ica-095-variance_compica.fif ...
Now restoring ICA solution ...
Ready.
Creating RawArray with float64 data, n_channels=25, n_times=30835
    Range : 0 ... 30834 =      0.000 ...   123.336 secs
Ready.
Using matplotlib as 2D backend.


C:\Users\User\AppData\Local\Temp\ipykernel_24936\1877083123.py:24: RuntimeWarning: This filename (rEEG\sub-095\ica-095-variance_compica.fif) does not conform to MNE naming conventions. All ICA files should end with -ica.fif, -ica.fif.gz, _ica.fif or _ica.fif.gz
  ica = read_ica(ICA_PATH)


Applying ICA to Raw instance
    Transforming to ICA space (25 components)
    Zeroing out 4 ICA components
    Projecting back using 60 PCA components
Overwriting existing file.
Writing c:\Users\User\Documents\EEG_Project\rEEG\sub-095\postICA095_raw.fif
Overwriting existing file.
Closing c:\Users\User\Documents\EEG_Project\rEEG\sub-095\postICA095_raw.fif
[done]
Saved cleaned raw → rEEG\sub-095\postICA095_raw.fif
Channels marked as bad:
none
Channels marked as bad:
none
Effective window size : 8.192 (s)
Effective window size : 8.192 (s)
Plotting power spectral density (dB=True).
Plotting power spectral density (dB=True).


In [2]:
import numpy as np
import mne
import matplotlib.pyplot as plt

# raw = mne.io.read_raw_fif('rEEG\sub-047\preICA047_raw.fif')
# raw_clean = mne.io.read_raw_fif('rEEG\sub-047\postICA047_raw.fif')

def compute_1_f_slope(raw, fmin=2, fmax=40):
    psd = raw.compute_psd(fmin=fmin, fmax=fmax, method="welch")
    freqs = psd.freqs
    psd_mean = psd.get_data().mean(axis=0)

    log_f = np.log10(freqs)
    log_p = np.log10(psd_mean)

    slope, intercept = np.polyfit(log_f, log_p, 1)
    return slope, freqs, psd_mean

slope_before, f, p_before = compute_1_f_slope(raw)
slope_after, _, p_after = compute_1_f_slope(raw_clean)

print(f"1/f slope BEFORE ICA: {slope_before:.2f}")
print(f"1/f slope AFTER  ICA: {slope_after:.2f}")

plt.figure()
plt.loglog(f, p_before, label="Before ICA")
plt.loglog(f, p_after, label="After ICA")
plt.legend()
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.title("PSD (1/f check)")
plt.show()


Effective window size : 8.192 (s)
Effective window size : 8.192 (s)
1/f slope BEFORE ICA: -1.04
1/f slope AFTER  ICA: -0.80


In [3]:
import mne
import numpy as np
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt

# -------------------------
# CHANNEL PICKS
# -------------------------
occipital = ["O1", "O2", "Oz"]

# -------------------------
# FUNCTIONS
# -------------------------
def alpha_power(raw, picks):
    psd = raw.compute_psd(
        picks=picks,
        fmin=1,
        fmax=30,
        method="welch"
    )

    freqs = psd.freqs
    data = psd.get_data()        # shape: (n_channels, n_freqs)
    psd_mean = data.mean(axis=0)

    alpha_mask = (freqs >= 8) & (freqs <= 13)
    alpha_peak = psd_mean[alpha_mask].max()

    return freqs, psd_mean, alpha_peak

# -------------------------
# RUN CHECK
# -------------------------
f, psd_before, alpha_before = alpha_power(raw, occipital)
_, psd_after, alpha_after = alpha_power(raw_clean, occipital)

print(f"Alpha peak BEFORE ICA: {alpha_before:.2e}")
print(f"Alpha peak AFTER ICA:  {alpha_after:.2e}")

# -------------------------
# PLOT
# -------------------------
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(f, psd_before)
plt.title("Occipital PSD – BEFORE ICA")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")

plt.subplot(1, 2, 2)
plt.plot(f, psd_after)
plt.title("Occipital PSD – AFTER ICA")
plt.xlabel("Frequency (Hz)")

plt.tight_layout()
plt.show()


Effective window size : 8.192 (s)
Effective window size : 8.192 (s)
Alpha peak BEFORE ICA: 4.09e-13
Alpha peak AFTER ICA:  5.07e-14


In [4]:
import numpy as np
import matplotlib.pyplot as plt

# Occipital channels
occipital = ["O1", "O2", "Oz"]

def alpha_integrity(raw, picks):
    psd = raw.compute_psd(picks=picks, fmin=1, fmax=30, method="welch")
    freqs = psd.freqs
    data = psd.get_data().mean(axis=0)

    # Bands
    theta = (freqs >= 4) & (freqs < 8)
    alpha = (freqs >= 8) & (freqs <= 13)

    # Metrics
    alpha_peak_freq = freqs[alpha][np.argmax(data[alpha])]
    alpha_rel_power = data[alpha].sum() / data.sum()
    theta_alpha_ratio = data[theta].sum() / data[alpha].sum()

    return freqs, data, alpha_peak_freq, alpha_rel_power, theta_alpha_ratio

# Run check
f, psd_before, peak_b, rel_b, tar_b = alpha_integrity(raw, occipital)
_, psd_after, peak_a, rel_a, tar_a = alpha_integrity(raw_clean, occipital)

# Print metrics
print("Alpha integrity check:")
print(f"Peak freq BEFORE: {peak_b:.2f} Hz | AFTER: {peak_a:.2f} Hz")
print(f"Relative alpha BEFORE: {rel_b:.3f} | AFTER: {rel_a:.3f}")
print(f"Theta/Alpha ratio BEFORE: {tar_b:.2f} | AFTER: {tar_a:.2f}")

# Plot shape only
plt.figure(figsize=(6, 4))
plt.plot(f, psd_before, label="Before ICA")
plt.plot(f, psd_after, label="After ICA")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.title("Occipital PSD Shape Check")
plt.legend()
plt.tight_layout()
plt.show()


Effective window size : 8.192 (s)
Effective window size : 8.192 (s)
Alpha integrity check:
Peak freq BEFORE: 10.62 Hz | AFTER: 11.84 Hz
Relative alpha BEFORE: 0.147 | AFTER: 0.133
Theta/Alpha ratio BEFORE: 1.01 | AFTER: 1.24
